In [1]:
import nltk
import string
import numpy as np
import operator
import math
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer


def normalize_line(line):
    tokenizer = RegexpTokenizer(r'\w+')
    stemmer = SnowballStemmer("english")
    words = [stemmer.stem(word) for word in tokenizer.tokenize(line) if word not in stopwords.words('english')]
    return words


documents = {}
N = 0
with open('cran.all.1400', 'r') as f:
    line = f.readline()
    while line:
        document = {}
        document_id = line.split()[1]
    
        f.readline()  # skip .T line

        line = f.readline()
        title_words = []
        while not line.startswith('.A'):
            title_words.extend(normalize_line(line))
            line = f.readline()
        document['title'] = title_words

        line = f.readline()
        while not line.startswith('.W'):
            line = f.readline()

        line = f.readline()
        annotation_words = []
        while not line.startswith('.I') and line:
            annotation_words.extend(normalize_line(line))
            line = f.readline()
        document['annotation'] = annotation_words

        documents[document_id] = document
        N += 1   
    
L_title = 0  # average length of title in documents
for document_index, document in documents.iteritems():
    L_title += len(document['annotation'])
    L_title /= float(N)


L_annotation = 0  # average length of annotation in documents
for document_index, document in documents.iteritems():
    L_annotation += len(document['annotation'])
    L_annotation /= float(N)


In [2]:
def create_inverted_index(document_field_name):
    invert_index = {}
    for document_index, document in documents.iteritems():
        for word in document[document_field_name]:
            if word in invert_index.keys() and document_index in invert_index[word].keys():
                pass
            else:
                if word not in invert_index:
                    invert_index[word] = {document_index: document[document_field_name].count(word)}
                else:
                    invert_index[word][document_index] = document[document_field_name].count(word)

    return invert_index

In [44]:
def search_in_index(index, query_words, field_name, b, k1, rsv_function_type, k2):
    founded_documents_indexes = set()
    for query_word in query_words:
        if query_word in index:
            founded_documents_indexes.update(index[query_word].keys())

    documents_with_rsv = {}
    for founded_document in founded_documents_indexes:
        document_rsv = rsv(query_words, documents[founded_document][field_name], index, field_name, b, k1, rsv_function_type, k2)
        documents_with_rsv[founded_document] = document_rsv

    return documents_with_rsv


def rsv(query_words, document_words, inverse_index, field_name, b, k1, rsv_function_type, k2):
    if field_name == 'title':
        L = L_title
    else:
        L = L_annotation

    rsv = 0
    for query_word in query_words:
        if query_word in inverse_index:
            Nt = sum(inverse_index[query_word].values())
            ftd = document_words.count(query_word)
            ftq = query_words.count(query_word)
            Ld = len(document_words)
            idf_sum = 0
            if rsv_function_type == 1:
                rsv += math.log1p(1 + (N - Nt + 0.5) / (Nt + 0.5)) * ftd * (k1 + 1)/(k1 * ((1 - b) + b * Ld / L) + ftd)
            elif rsv_function_type == 2:  # without -Nt
                rsv += math.log1p(1 + (N + 0.5) / (Nt + 0.5)) * ftd * (k1 + 1)/(k1 * ((1 - b) + b * Ld / L) + ftd)
            elif rsv_function_type == 3:  # normalized
                rsv += math.log1p(1 + (N - Nt + 0.5) / (Nt + 0.5)) * ftd * (k1 + 1)/(k1 * ((1 - b) + b * Ld / L) + ftd)
                idf_sum += math.log1p(1 + (N - Nt + 0.5) / (Nt + 0.5))
            elif rsv_function_type == 4:  # BM25 with k2
                rsv += math.log1p(1 + (N - Nt + 0.5) / (Nt + 0.5)) * ftd * (k1 + 1)/(k1 * ((1 - b) + b * Ld / L) + ftd) *\
                (k2+1) * ftq /(k2+ftq)
                
    if rsv_function_type == 3:
        rsv /= float(idf_sum)
        
    return rsv

In [3]:
title_index = create_inverted_index('title')
annotation_index = create_inverted_index('annotation')

In [32]:
def answer_for_field(field_name, b=0.75, k1=1.2, rsv_function_type=1, k2=0, rsv_threshold = 0):
    with open('answer', 'w') as result_file:
        with open('cran.qry', 'r') as f:
            line = f.readline()
            query_number = 0
            while line:
                line = f.readline()  # skip .W line 

                query_words = []
                while not line.startswith('.I') and line:
                    query_words.extend(normalize_line(line))
                    line = f.readline()

                documents_with_rsv = search_in_index(title_index, query_words, field_name, b, k1, rsv_function_type, k2)
                top10 = [elem[0] for elem in
                         sorted(documents_with_rsv.items(), key=operator.itemgetter(1), reverse=True)[:10] if elem[1] > rsv_threshold]

                query_number += 1
                for top in top10:
                    result_file.write('%s %s\n' % (query_number, top))


In [33]:
answer_for_field('title')
%run ./eval.py


mean precision: 0.220888888889
mean recall: 0.323529928296
mean F-measure: 0.262533785122
MAP@10: 0.2319249986


In [45]:
answer_for_field('annotation')
%run ./eval.py

mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.285417813751


Поиск по индексу, построенному по аннатациям получился более точным. Так как аннотации больше по размеру и там больше информации.

In [21]:
def eval():
    groundtruth_file = 'qrel_clean'
    answer_file = 'answer'
    
    q2reld = {} 
    for line in open(groundtruth_file):
        qid, did = [int(x) for x in line.split()]
        if qid not in q2reld.keys():
            q2reld[qid] = set()
        q2reld[qid].add(did)        
    
    q2retrd = {}
    for line in open(answer_file):
        qid, did = [int(x) for x in line.split()]
        if qid not in q2retrd.keys():
            q2retrd[qid] = []
        q2retrd[qid].append(did)               
    
    N = len(q2retrd.keys())
    precision = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2retrd[q]) for q in q2retrd.keys()]) / N
    recall = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2reld[q]) for q in q2retrd.keys()]) / N
    print("mean precision: {}\nmean recall: {}\nmean F-measure: {}"\
          .format(precision, recall, 2*precision*recall/(precision+recall)))

    # MAP@10
    import numpy as np
    
    MAP = 0.0
    for q in q2retrd.keys():
        n_results = min(10, len(q2retrd[q]))
        avep = np.zeros(n_results)
        for i in range(n_results):
            avep[i:] += q2retrd[q][i] in q2reld[q]
            avep[i] *= (q2retrd[q][i] in q2reld[q]) / (i+1.0)
        MAP += sum(avep) / min(n_results, len(q2reld[q]))
    print("MAP@10: {}".format(MAP/N))
    return MAP/N


In [22]:
best_result = 0
for k in np.arange(1.2, 2.1, 0.1):
    for b in np.arange(0, 1.1, 0.1):
        answer_for_field('annotation', b, k)
        eval_result = eval()
        if eval_result > best_result:
            best_result = eval_result
            best_params = (k, b)

mean precision: 0.250222222222
mean recall: 0.373188082114
mean F-measure: 0.299577823991
MAP@10: 0.285166690462
mean precision: 0.250222222222
mean recall: 0.368296413107
mean F-measure: 0.297989233178
MAP@10: 0.286135901151
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.285922443381
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.28558558271
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.285532672658
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.285285759077
mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.285440035973
mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.285417813751
mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.285

mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.285406055961
mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.285406055961
mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.285449265838
mean precision: 0.253777777778
mean recall: 0.376361723288
mean F-measure: 0.303146340184
MAP@10: 0.285052386551
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.285883289941
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.285479762605
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.285448677949
mean precision: 0.250666666667
mean recall: 0.368518635329
mean F-measure: 0.298377037132
MAP@10: 0.285226455726
mean precision: 0.251111111111
mean recall: 0.36925937607
mean F-measure: 0.29893469831
MAP@10: 0.2854

In [24]:
print best_result
print best_params

answer_for_field('annotation', 0, 1.3)
eval_result = eval()

0.287127184289
(1.3, 0.0)
mean precision: 0.252444444444
mean recall: 0.376109327535
mean F-measure: 0.302111655272
MAP@10: 0.287127184289


В рузультате подбора параметров с помощью grid search, получилось, что лучший результат по MAP10 при k1 = 1.3 и b=0
Т.к. параметр b отвечает за важность длины документа, то видимо длина документа не важно для данного примера.

Попробуйте заменить функцию вычисления IDF-составляющей в формуле BM25 выше на один из других вариантов, представленных на лекции

Для этого убрал Nt из числителя под лагорифмом (так было сказано на лекции)

In [25]:
best_result = 0
for k in np.arange(1.2, 1.5, 0.1):
    for b in np.arange(0, 1.1, 0.1):
        answer_for_field('annotation', b, k, 2)
        eval_result = eval()
        if eval_result > best_result:
            best_result = eval_result
            best_params = (k, b)
            
print best_params
print best_result

mean precision: 0.250222222222
mean recall: 0.375685779393
mean F-measure: 0.300379385898
MAP@10: 0.286574104868
mean precision: 0.252888888889
mean recall: 0.373572894069
mean F-measure: 0.301606376223
MAP@10: 0.287880112539
mean precision: 0.251555555556
mean recall: 0.371517338514
mean F-measure: 0.299988175952
MAP@10: 0.28667111783
mean precision: 0.250666666667
mean recall: 0.370665486662
mean F-measure: 0.299078299722
MAP@10: 0.28634483917
mean precision: 0.250666666667
mean recall: 0.370665486662
mean F-measure: 0.299078299722
MAP@10: 0.286076291257
mean precision: 0.250666666667
mean recall: 0.370665486662
mean F-measure: 0.299078299722
MAP@10: 0.286069432547
mean precision: 0.250666666667
mean recall: 0.370665486662
mean F-measure: 0.299078299722
MAP@10: 0.286069432547
mean precision: 0.250666666667
mean recall: 0.370665486662
mean F-measure: 0.299078299722
MAP@10: 0.286080896391
mean precision: 0.250666666667
mean recall: 0.370665486662
mean F-measure: 0.299078299722
MAP@10: 

Результат получился лучше

Попробуем нормализовать RSV (q, d) на сумму IDF термов запроса

In [27]:
answer_for_field('annotation', 0, 1.3, 3)
eval()

mean precision: 0.252444444444
mean recall: 0.376109327535
mean F-measure: 0.302111655272
MAP@10: 0.287127184289


0.28712718428935369

С нормализацией результат таким же как и без нормализации

На лекции был предложен общий вариант BM25, включающий так-
(k +1)f
же множитель k 2 2 +f t,q t,q . Добавьте его в формулу вычисления RSV
и исследуйте вопрос оптимальности параметра k 2 (k 2 может варьи-
роваться от 0 до 1000 – тут скорее важен оптимальный порядок
константы)

In [28]:
best_result = 0
best_k2 = 0
for k2 in range(1, 1000, 50):
    answer_for_field('annotation', 0.75, 1.2, 4, k2)
    eval_result = eval()
    if eval_result > best_result:
        best_result = eval_result
        best_k2 = k2

mean precision: 0.248444444444
mean recall: 0.365617832428
mean F-measure: 0.295851813986
MAP@10: 0.280462990958
mean precision: 0.244444444444
mean recall: 0.358474975285
mean F-measure: 0.290676376688
MAP@10: 0.270423780829
mean precision: 0.244444444444
mean recall: 0.358474975285
mean F-measure: 0.290676376688
MAP@10: 0.270144768484
mean precision: 0.244444444444
mean recall: 0.358474975285
mean F-measure: 0.290676376688
MAP@10: 0.269589212928
mean precision: 0.244
mean recall: 0.357586086396
mean F-measure: 0.290069890424
MAP@10: 0.269252175891
mean precision: 0.244
mean recall: 0.357586086396
mean F-measure: 0.290069890424
MAP@10: 0.269252175891
mean precision: 0.244
mean recall: 0.357586086396
mean F-measure: 0.290069890424
MAP@10: 0.269252175891
mean precision: 0.244
mean recall: 0.357586086396
mean F-measure: 0.290069890424
MAP@10: 0.269252175891
mean precision: 0.244
mean recall: 0.357586086396
mean F-measure: 0.290069890424
MAP@10: 0.269252175891
mean precision: 0.244
mean r

In [29]:
print best_result
print best_k2

0.280462990958
1


Лучший результат получился при k2 = 1, т.е. при увеличении порядка k2 результат ухудшается. По идее без вообще без него результат получается лучше.

Не всегда количество релевантных запросу документов достигает 10. Можно ли установить порог на значение RSV документа, включаемого в результат поиска, такой, что это улучшит precision?

Да, можно. Т.к. за счёт порога документы с низким rcv не будут попадать в результаты поиска, а их там и не должно быть